## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,84.92,54,100,1.01,overcast clouds
1,1,Marzuq,YE,14.4000,46.4667,75.00,16,31,3.44,scattered clouds
2,2,Hermanus,ZA,-34.4187,19.2345,58.53,77,6,5.77,clear sky
3,3,Rikitea,PF,-23.1203,-134.9692,76.15,68,0,22.79,clear sky
4,4,Hay River,CA,60.8156,-115.7999,53.87,59,100,9.15,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,84.92,54,100,1.01,overcast clouds
37,37,Takoradi,GH,4.8845,-1.7554,80.62,83,100,7.40,light rain
44,44,Lorengau,PG,-2.0226,147.2712,80.47,74,65,2.82,broken clouds
65,65,Palmas,BR,-10.2128,-48.3603,87.67,62,20,3.44,light rain
80,80,Presidente Medici,BR,-11.1753,-61.9014,88.36,57,8,1.66,clear sky
86,86,Witu,KE,-2.3889,40.4382,80.76,85,100,9.73,overcast clouds
89,89,Hithadhoo,MV,-0.6000,73.0833,83.10,68,93,9.42,overcast clouds
90,90,Natal,BR,-5.7950,-35.2094,83.05,83,75,12.66,broken clouds
92,92,Butaritari,KI,3.0707,172.7902,80.44,75,27,8.50,scattered clouds
100,100,Monteria,CO,8.7480,-75.8814,85.23,75,100,5.70,overcast clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,84.92,overcast clouds,42.6001,-73.9662,
37,Takoradi,GH,80.62,light rain,4.8845,-1.7554,
44,Lorengau,PG,80.47,broken clouds,-2.0226,147.2712,
65,Palmas,BR,87.67,light rain,-10.2128,-48.3603,
80,Presidente Medici,BR,88.36,clear sky,-11.1753,-61.9014,
86,Witu,KE,80.76,overcast clouds,-2.3889,40.4382,
89,Hithadhoo,MV,83.10,overcast clouds,-0.6000,73.0833,
90,Natal,BR,83.05,broken clouds,-5.7950,-35.2094,
92,Butaritari,KI,80.44,scattered clouds,3.0707,172.7902,
100,Monteria,CO,85.23,overcast clouds,8.7480,-75.8814,


In [34]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,84.92,overcast clouds,42.6001,-73.9662,
37,Takoradi,GH,80.62,light rain,4.8845,-1.7554,Raybow International Hotel
44,Lorengau,PG,80.47,broken clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
65,Palmas,BR,87.67,light rain,-10.2128,-48.3603,Hotel Lago da Palma
80,Presidente Medici,BR,88.36,clear sky,-11.1753,-61.9014,Oliveira Palace Hotel


In [36]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
37,Takoradi,GH,80.62,light rain,4.8845,-1.7554,Raybow International Hotel
44,Lorengau,PG,80.47,broken clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
65,Palmas,BR,87.67,light rain,-10.2128,-48.3603,Hotel Lago da Palma
80,Presidente Medici,BR,88.36,clear sky,-11.1753,-61.9014,Oliveira Palace Hotel
89,Hithadhoo,MV,83.10,overcast clouds,-0.6000,73.0833,Scoop Guest House
90,Natal,BR,83.05,broken clouds,-5.7950,-35.2094,Villa Park Hotel | Natal
92,Butaritari,KI,80.44,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
100,Monteria,CO,85.23,overcast clouds,8.7480,-75.8814,Hotel Florida Sinú
102,Hilo,US,80.28,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
105,Touros,BR,82.80,overcast clouds,-5.1989,-35.4608,INN NEW HORIZON


In [37]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
